In [1]:
# block 1st
#upload raw data
import pandas as pd
raw_data = pd.read_csv("https://raw.githubusercontent.com/naynco/nayn.data/master/classification_clean.csv")

In [5]:
#In case URL is dead
#Back-up for for raw_data
#import pickle
#raw_data.to_pickle("./raw_data.pkl")
#For backing up saved raw_data
#with open("raw_data.pkl", "rb") as fp:
#    raw_data= pickle.load(fp)


In [2]:
#block 2nd

#learn size of raw_data
#print(len(raw_data))
#learn unique values in Categories column
##raw_data["Categories"].unique()
#tackled categories
main_categories = ['DÜNYA', 'SPOR','SANAT','Teknoloji']
#get filter
filter=raw_data["Categories"].isin(main_categories)
#apply filter
filtered_data=raw_data[filter]
#learn size of filtered_data
#print(len(filtered_data))
##filtered_data.head()
#remove redundant columns
tackled_data0=filtered_data[['Title','Categories']]
tackled_data0.head()

,Title,Categories
12006,58 Saniyede Katar Meselesi? Katar krizi nedir?...,DÜNYA
12496,58 Saniyede Türkiye - Almanya Gerginliği,DÜNYA
12877,"Adriana Lima, Bomba Aşkla İlgili İlk Kez Konuş...",DÜNYA
12878,Galatasaraylı Taraftarlar Patladı: İstifa Edin,SPOR
12880,"Galatasaray'dan Ayrılan Sabri, Neredeyse Bedav...",SPOR


In [5]:
#block 3rd

#method-just lower case as a transformation
from unicode_tr import unicode_tr
# just lower case
def just_lower(doc):
    doc=unicode_tr(doc).lower()
    return doc
tackled_data = tackled_data0.copy()
#
tackled_data['Title'] = tackled_data0['Title'].apply(just_lower)
y_general=tackled_data["Categories"]

#tackled_data[tackled_data.index==14252]
tackled_data.head()

,Title,Categories
12006,58 saniyede katar meselesi? katar krizi nedir?...,DÜNYA
12496,58 saniyede türkiye - almanya gerginliği,DÜNYA
12877,"adriana lima, bomba aşkla ilgili ilk kez konuş...",DÜNYA
12878,galatasaraylı taraftarlar patladı: istifa edin,SPOR
12880,"galatasaray'dan ayrılan sabri, neredeyse bedav...",SPOR


In [6]:
#block 4th
#split train-test datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tackled_data['Title'],tackled_data['Categories'],test_size=0.2,random_state=57)

In [7]:
#block 5th
#save y_test_57
y_test.to_pickle("./y_test_57.pkl")

#save y_train_57
y_train.to_pickle("./y_train_57.pkl")


In [8]:
#block 6th
#get distribution of categories in general, train and test set, respectively
general_counts=[0,0,0,0]
general_percents=[.0,.0,.0,.0]
train_counts=[0,0,0,0]
train_percents=[.0,.0,.0,.0]
test_counts=[0,0,0,0]
test_percents=[.0,.0,.0,.0]
general_count=len(y_general)
train_count=len(y_train)
test_count=len(y_test)
for i in range(0,4):
    general_counts[i]=y_general[y_general==main_categories[i]].count()
    general_percents[i]=round(general_counts[i]/general_count,2)
    train_counts[i]=y_train[y_train==main_categories[i]].count()
    train_percents[i]=round(train_counts[i]/train_count,2)
    test_counts[i]=y_test[y_test==main_categories[i]].count()
    test_percents[i]=round(test_counts[i]/test_count,2)
    
#print("summary")
#print(main_categories)
#print("for general")
#print(general_counts) 
#print(general_percents) 
#print("for train")
#print(train_counts) 
#print(train_percents)
#print("for test")
#print(test_counts) 
#print(test_percents)

In [9]:
#block 7th
#Primitive sub_methods for analysis
# the method saying whether word is in document
def is_inside(word,doc):
    if doc.find(word)==-1:
        return False
    else:
        return True
# the method saying whether word list is in document together    
def is_inside_list(word_list,doc):
    result=True
    for word in word_list:
        result=(result and is_inside(word,doc))
        
    return result


In [10]:
#block 8th
#test for methods in block 7th
#from first part of poland anthem
#print(is_inside("żyjemy","Jeszcze Polska nie zginęła,Kiedy my żyjemy.Co nam obca przemoc wzięła,Szablą odbierzemy."))
#from first part of estonian anthem
#print(is_inside_list(["kaunis","isamaa"],"Mu isamaa, mu õnn ja rõõm,kui kaunis oled sa!Ei leia mina iial teal"))
#from first part of poland anthem
#print(is_inside("Przejdziem","Jeszcze Polska nie zginęła,Kiedy my żyjemy.Co nam obca przemoc wzięła,Szablą odbierzemy."))
#from first part of estonian anthem
#print(is_inside_list(["kaunis","truuiks"],"Mu isamaa, mu õnn ja rõõm,kui kaunis oled sa!Ei leia mina iial teal"))

In [11]:
#block 9th
#analyze methods
#analyze_word, get weights to indicate tendencies of a word over categories
def analyze_word(word,train_X,train_y):
    #initialization of output, count of word in docs with category
    category_counts=[0,0,0,0]
    #iterate over train_X
    for i in train_X.index:
            doc=train_X[i]
            #word in doc ?
            if is_inside(word,doc):
                #if yes, what category of the doc?
                for j in range(0,4):
                    #if yes, increment by 1 on count of related category
                    if train_y[i]==main_categories[j]:
                        category_counts[j]=category_counts[j]+1
                
                 
    #sum of counts of number of category                   
    sum_counts=sum(category_counts)
    #if there is no category related to the word, then no prediction
    if sum_counts==0:
            sum_counts=1
            label1="No Prediction"
            label2="No Prediction"
    else:
        temp=category_counts.copy()
        #get category having the largest count of word on docs
        label1=main_categories[temp.index(max(temp))]
        #get category having 2nd largest count of word on docs
        #(1)set 0 for category having the largest count of word on docs
        temp[temp.index(max(temp))]=0
        #(2)thereby,category of largest count of temp after setting is category having 2nd largest count of word on docs
        label2=main_categories[temp.index(max(temp))]
        #if there is no 2nd largest count of word on docs
        if max(temp)==0:
            label2="No Prediction"
        else:
            label2=main_categories[temp.index(max(temp))]
     
    #get rates, count by sum of counts, as probabilities of categories over word 
    category_probs = [x / sum_counts for x in category_counts]
    #round to two digits
    category_probs=[round(x,2) for x in category_probs ]
    #at last outputs: the word, lengths of the word, category counts, label1 ans label2 
    return [word,len(word),category_counts,category_probs,label1,label2]

#analyze_wordlist, get weights to indicate tendencies of elements of wordlist over categories
def analyze_wordlist(word_list,train_X,train_y):
   #initialization of output, count of word in docs with category
    category_counts=[0,0,0,0]
    #iterate over train_X
    for i in train_X.index:
            doc=train_X[i]
            #elements of word_list in doc ?
            if is_inside_list(word_list,doc):               
                for j in range(0,4):
                    #if yes, increment by 1 on count of related category
                    if train_y[i]==main_categories[j]:                        
                        category_counts[j]=category_counts[j]+1
                       
               
    #sum of counts of number of category                   
    sum_counts=sum(category_counts)
    #if there is no category related to the word, then no prediction
    if sum_counts==0:
            sum_counts=1
            label1="No Prediction"
            label2="No Prediction"
    else:
        temp=category_counts.copy()
        #get category having the largest count of word on docs
        label1=main_categories[temp.index(max(temp))]
        #get category having 2nd largest count of word on docs
        #(1)set 0 for category having the largest count of word on docs
        temp[temp.index(max(temp))]=0
        #(2)thereby,category of largest count of temp after setting is category having 2nd largest count of word on docs
        label2=main_categories[temp.index(max(temp))]
        #if there is no 2nd largest count of word on docs
        if max(temp)==0:
            label2="No Prediction"
        else:
            label2=main_categories[temp.index(max(temp))]
    #get rates, count by sum of counts, as probabilities of categories over word 
    category_probs = [x / sum_counts for x in category_counts]
    #round to two digits
    category_probs=[round(x,2) for x in category_probs ]
    #at last outputs: word_list, sum of lengths of words in wordlist, category counts, label1 ans label2 
    return [word_list,len(''.join(word_list)),category_counts,category_probs,label1,label2]

#extended(detailed) analyze methods in case of need
#%run ./extended_analysis_methods.ipynb


In [12]:
#block 10th
#test for analyze_word
#print(analyze_word("finans",X_train,y_train))
#print(analyze_word("galatasaray",X_train,y_train))
#print(analyze_word("matmazel",X_train,y_train))
#print(analyze_word("oscar",X_train,y_train))
#print(analyze_word("filistin",X_train,y_train))
#print(analyze_word("israil",X_train,y_train))
#print(analyze_word("dolar",X_train,y_train))
#print(analyze_word("euro",X_train,y_train))
#test for analyze_wordlist
#print(analyze_wordlist(["filistin","israil"],X_train,y_train))
#print(analyze_wordlist(["filistin","israil","abd"],X_train,y_train))
#print(analyze_wordlist(["oscar","oyuncu"],X_train,y_train))

In [13]:
#block 9th
#import preprocess methods
%run ./Step2_preprocess_word.ipynb

In [14]:
#block 10th
#test preprocess method
#import random
#n=random.randint(1,test_count)
#print(n)
#doc=X_test[X_test.index[n]]
#label=y_test[X_test.index[n]]
#print("doc:",doc)
#print("label:",label)
#print(preprocess(doc))

In [15]:
#block 11th

#analyze methods for doc (string)

# one word-based

def analyze_doc(doc,train_X,train_y):
    #get stems with maximum length of words in doc
    max_stems=preprocess(doc)
    results=[]
    for word in max_stems:
        #analyze for each word in doc
        data=analyze_word(word,train_X,train_y)
        #if max_stems in docs of X_train
        if sum(data[2])!=0:
            #append to results
            results.append(data)
    
    
    
    return results

# wordlist with n-word selecting by combinational approach
#in order to select stems in doc, we use a method based on combination
#for example number of elements in list of preprocess(doc) is m=7, and you choose a combination with n=2 elements
#thereby you have C(7,2)=(7*6)/2=21 combinations as a wordlist with 2 elements.
#method gives combination as wordlist having tendencies over categories

import itertools

def analyze_doc_s(doc,train_X,train_y,s):
    #get stems with maximum length of words in doc
    max_stems=preprocess(doc)
    #n, number of combination
    #len(max_stems) is number of element of preprocess(doc)
    # of course, if n>m, C(m,s) is not meaningful
    if s>len(max_stems):
        return []
    results=[]
    #s_list is list of combination with s element of preprocess(doc) or max_stems 
    s_list=list(set(itertools.combinations(max_stems, s)))
    for word_list in s_list:
        #for each combination as a wordlist, get result of method analyze_wordlist
        word_list=list(word_list)
        data=analyze_wordlist(word_list,train_X,train_y)
        #if that combination has at least one category, append the results
        # doc in test_set by default
        if sum(data[2])>0:
            results.append(data)
        
        #if doc in train_set
        # sum of count of categories must be greater than 1, because we want another documents involving that combination
        #if sum(data[2])>1:
            #results.append(data)
    
    
    
    return results

In [16]:
#block 12th
#doc defined in block 10th
#test of analyze_doc
#print("doc:",doc)
#print("label:",label)
#analyze_doc(doc,X_train,y_train)




In [17]:
#block 13th

#doc defined in block 10th
#test of analyze_doc_n
#analyze_doc_n(doc,X_train,y_train,2)